Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the dataframe:

    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples of how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use the BeautifulSoup package to transform the data in the table on the Wikipedia page into the above pandas dataframe

In [21]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests

In [22]:
!pip install bs4

  Using cached https://files.pythonhosted.org/packages/21/0a/47fdf541c97fd9b6a610cb5fd518175308a7cc60569962e776ac52420387/beautifulsoup4-4.6.3-py3-none-any.whl
Target directory /home/spark/shared/user-libs/python3/bs4-0.0.1.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/bs4 already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/beautifulsoup4-4.6.3.dist-info already exists. Specify --upgrade to force replacement.


In [23]:
from bs4 import BeautifulSoup as bs

In [24]:
from urllib.request import (
    urlopen, urlparse, urlunparse, urlretrieve)
from bs4 import BeautifulSoup as bs

In [25]:
import urllib.request

opener = urllib.request.FancyURLopener({})
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
f = opener.open(url)
content = f.read()

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  app.launch_new_instance()


In [26]:
# using beautiful soup to parse the HTML/XML codes
soup = bs(content,'xml')

In [27]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            #tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
# Ignore cells with a Borough that is Not assigned or None        
    if (Borough_var == 'Not assigned'): 
        continue
    try:
        if (tag_a_Borough is None):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [28]:
unique_p = set(Postcode)
print('Unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

Unique Postal codes: 100


Create dataframe

In [36]:
toronto_d = {'Postal Code':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df = pd.DataFrame.from_dict(toronto_d)
df.head(100)

,Borough,Neighbourhood,Postal Code
0,North York,Willowdale South,M2N
1,Central Toronto,Davisville North,M4P
2,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",M8Z
3,Etobicoke,Westmount,M9P
4,East Toronto,The Beaches,M4E
5,North York,Downsview Northwest,M3N
6,Central Toronto,"The Annex, North Midtown, Yorkville",M5R
7,Scarborough,Woburn,M1G
8,Downtown Toronto,Central Bay Street,M5G
9,Etobicoke,Northwest,M9W


In [37]:
df.sort_values(by=['Borough'])

,Borough,Neighbourhood,Postal Code
98,Central Toronto,Lawrence Park,M4N
42,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V
46,Central Toronto,"Forest Hill North, Forest Hill West",M5P
89,Central Toronto,North Toronto West,M4R
6,Central Toronto,"The Annex, North Midtown, Yorkville",M5R
40,Central Toronto,Roselawn,M5N
62,Central Toronto,Davisville,M4S
56,Central Toronto,"Moore Park, Summerhill East",M4T
1,Central Toronto,Davisville North,M4P
32,Downtown Toronto,"Commerce Court, Victoria Hotel",M5L


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [38]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough

In [39]:
df.sort_values(by=['Borough'])

,Borough,Neighbourhood,Postal Code
98,Central Toronto,Lawrence Park,M4N
42,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V
46,Central Toronto,"Forest Hill North, Forest Hill West",M5P
89,Central Toronto,North Toronto West,M4R
6,Central Toronto,"The Annex, North Midtown, Yorkville",M5R
40,Central Toronto,Roselawn,M5N
62,Central Toronto,Davisville,M4S
56,Central Toronto,"Moore Park, Summerhill East",M4T
1,Central Toronto,Davisville North,M4P
32,Downtown Toronto,"Commerce Court, Victoria Hotel",M5L


In [40]:
df.shape

(100, 3)

In [41]:
!wget -O geospatial.csv http://cocl.us/Geospatial_data

--2018-10-30 12:28:53--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2018-10-30 12:28:54--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-10-30 12:28:55--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2018-10-30 12:28:56--  https://ibm.ent.box.com/share

In [42]:
dfg = pd.read_csv('geospatial.csv')
dfg.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
dfn=pd.merge(df, dfg, on='Postal Code')

In [49]:
dfn=dfn[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]
dfn

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M2N,North York,Willowdale South,43.770120,-79.408493
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M3N,North York,Downsview Northwest,43.761631,-79.520999
6,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
7,M1G,Scarborough,Woburn,43.770992,-79.216917
8,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
9,M9W,Etobicoke,Northwest,43.706748,-79.594054
